##### 1. Utility Functions and Imports

In [ ]:
import os
import sys
import random
import shutil
from collections import Counter
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import EfficientNetV2B0, InceptionV3
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score
from keras.saving import register_keras_serializable
from tensorflow.keras.applications import vgg16, resnet50, inception_v3

In [ ]:
!pip install -q scikit-learn

#############
# Constants
#############

SEED=123
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 3
CLASS_NAMES = ['brain_glioma', 'brain_menin', 'brain_tumor']

# Update to True to use Drive for data storage
USE_DRIVE = False

#############
# Colab check
#############

def is_colab():
    return 'google.colab' in sys.modules

#############
# EDA
#############

def plot_raw_dataset(raw_data_dir):

    class_counts = {}

    for class_name in sorted(os.listdir(raw_data_dir)):
        class_path = os.path.join(raw_data_dir, class_name)
        if os.path.isdir(class_path):
            num_images = len([
                f for f in os.listdir(class_path)
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            ])
            class_counts[class_name] = num_images

    classes = list(class_counts.keys())
    counts = list(class_counts.values())

    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=classes, y=counts, palette=["#1f77b4", "#ff7f0e", "#2ca02c"])
    plt.title("Raw Dataset Class Distribution", fontsize=16)
    plt.xlabel("Number of Images")
    plt.ylabel("Class")
    for container in ax.containers:
        ax.bar_label(container)
    plt.tight_layout()
    plt.show()

def analyze_datasets(dataset, dataset_name="Dataset", class_names=None):

    print(f"{dataset_name} Analysis")
    print("="*50)

    total_samples = 0
    class_counts = Counter()
    all_labels = []

    for batch_images, batch_labels in dataset:
        batch_size = tf.shape(batch_images)[0].numpy()
        total_samples += batch_size

        if len(batch_labels.shape) > 1:
            batch_indices = tf.argmax(batch_labels, axis=1).numpy()
        else:
            batch_indices = batch_labels.numpy()

        all_labels.extend(batch_indices)

        for label in batch_indices:
            class_counts[int(label)] += 1

    all_labels = np.array(all_labels)

    return {
        'total_samples': total_samples,
        'class_counts': dict(class_counts),
        'all_labels': all_labels,
        'class_names': class_names
    }

def plot_class_distribution(stats, title="Class Distribution"):

    class_counts = stats['class_counts']
    class_names = stats['class_names']

    classes = list(class_counts.keys())
    counts = list(class_counts.values())

    if class_names:
        labels = [class_names[i] if i < len(class_names) else f"Class {i}" for i in classes]
    else:
        labels = [f"Class {i}" for i in classes]

    plt.figure(figsize=(10, 6))
    bars = plt.bar(labels, counts, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'][:len(classes)])

    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts)*0.01,
                f'{count:,}', ha='center', va='bottom', fontweight='bold')

    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel('Classes', fontsize=12)
    plt.ylabel('Number of Samples', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

def show_dataset_samples():

    train_dir = os.path.join(DATA_DIR, "Training")
    classes = CLASS_NAMES

    plt.figure(figsize=(8, 9))

    for i in range(3):
        for j in range(2):
            plt.subplot(3, 2, i*2 + j + 1)
            plt.xticks([])
            plt.yticks([])

            class_path = os.path.join(train_dir, classes[i])
            files = sorted([f for f in os.listdir(class_path)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

            if j < len(files):
                img_path = os.path.join(class_path, files[j])
                img = plt.imread(img_path)
                plt.imshow(img)
                plt.xlabel(f"{classes[i]}\n{files[j]}")

    plt.tight_layout()
    plt.show()

#################################################################
# Dataset preprocessing functions
#################################################################

def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=SEED):
    random.seed(seed)
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)

    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1."

    classes = [d.name for d in input_dir.iterdir() if d.is_dir()]

    for cls in classes:
        class_dir = input_dir / cls
        images = sorted(list(class_dir.glob('*')))
        random.shuffle(images)

        n_total = len(images)
        n_train = round(train_ratio * n_total)
        n_val = round(val_ratio * n_total)
        n_test = n_total - n_train - n_val

        splits = {
            'Training': images[:n_train],
            'Validation': images[n_train:n_train + n_val],
            'Testing': images[n_train + n_val:],
        }

        for split_name, split_files in splits.items():
            split_path = output_dir / split_name / cls
            split_path.mkdir(parents=True, exist_ok=True)
            for f in split_files:
                shutil.copy(f, split_path / f.name)

    print("Dataset successfully split into Training, Validation, and Testing folders.")


def load_images_from_directory(directory, image_size=IMAGE_SIZE):

    X = []
    y = []
    class_names = sorted(os.listdir(directory))

    for label in class_names:
        class_dir = os.path.join(directory, label)
        if not os.path.isdir(class_dir):
            continue
        for file in os.listdir(class_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(class_dir, file)
                try:
                    # img = Image.open(img_path).convert('L')
                    img = Image.open(img_path).convert('RGB')
                    img = img.resize(image_size)
                    X.append(np.array(img))
                    y.append(label)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")

    X = np.array(X, dtype='float32') / 255.0
    y = np.array(y)

    return X, y, class_names


def load_dataset_from_splits(base_dir, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, seed=SEED):

    X_train, y_train_raw, _ = load_images_from_directory(os.path.join(base_dir, "Training"), image_size)
    X_val, y_val_raw, _ = load_images_from_directory(os.path.join(base_dir, "Validation"), image_size)
    X_test, y_test_raw, _ = load_images_from_directory(os.path.join(base_dir, "Testing"), image_size)

    all_labels = np.concatenate([y_train_raw, y_val_raw, y_test_raw])
    label_encoder = LabelEncoder()
    label_encoder.fit(all_labels)

    y_train = label_encoder.transform(y_train_raw)
    y_val = label_encoder.transform(y_val_raw)
    y_test = label_encoder.transform(y_test_raw)

    assert len(X_train) == len(y_train), "Mismatch in train X/y"
    assert len(X_val) == len(y_val), "Mismatch in val X/y"
    assert len(X_test) == len(y_test), "Mismatch in test X/y"

    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))\
        .shuffle(buffer_size=len(X_train), seed=seed)\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)

    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))\
        .shuffle(buffer_size=len(X_val), seed=seed)\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)

    test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))\
        .batch(batch_size)\
        .prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds, label_encoder.classes_


#################################################################
# Model Plots - Evaluation Metrics
#################################################################

def evaluate_model(model, test_ds, class_names, model_name="Model"):

    y_true = []
    y_pred = []

    for images, labels in test_ds:
        preds = model.predict(images)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(labels.numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    acc = np.mean(y_true == y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')

    print(f"{model_name}")
    print(f"Accuracy : {acc:.3f}")
    print(f"F1 Score : {f1:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall   : {recall:.3f}")

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

    plt.figure(figsize=(6, 6))
    disp.plot(cmap=plt.cm.Blues, values_format="d")
    plt.title(f"Confusion Matrix: {model_name}")
    plt.show()

    return acc, f1, precision, recall


def plot_training_curves(history, title="Model"):

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{title} Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
#################################################################
# Model functions
#################################################################

def compile_and_fit(model,
                    train_data,
                    val_data,
                    model_name,
                    epochs=20,
                    lr=1e-4,
                    checkpoint_dir='./checkpoints'):

    os.makedirs(checkpoint_dir, exist_ok=True)
    ckpt_path = os.path.join(checkpoint_dir, f"{model_name}_best.h5")

    checkpoint_cb = callbacks.ModelCheckpoint(
        filepath=ckpt_path,
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        verbose=1
    )

    early_stop_cb = callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=5,
        mode="max",
        restore_best_weights=True
    )

    reduce_lr_cb = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_delta=1e-3,
        min_lr=1e-6,
        verbose=1
    )

    # target labels are integers (not one-hot encoded vectors) - more memory-efficient
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[checkpoint_cb, early_stop_cb, reduce_lr_cb]
    )

    return history, ckpt_path


def save_model_summary(model, model_name="model", save_dir="./model_summaries"):

    os.makedirs(save_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{model_name}_summary_{timestamp}.txt"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, "w") as f:
        model.summary(print_fn=lambda x: f.write(x + "\n"))

    print(f"Model summary saved to: {filepath}")

    return filepath


##### 2. Setup - Environment check - DataSet split

In [ ]:
if is_colab():
    print("Running in Google Colab")

    !pip install -q kaggle

    from google.colab import files
    kaggleKey = "/content/kaggle.json"
    if not os.path.exists(kaggleKey):
      key = files.upload()
    else:
      print("Kaggle.json exists")

    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    if USE_DRIVE:
    # Google Drive paths
      print("Using google Drive and persistent dataset")
      from google.colab import drive
      drive.mount('/content/drive')

      RAW_DATA_DIR = "/content/drive/MyDrive/datasets/brain_tumor_raw/Brain_Cancer raw MRI data/Brain_Cancer"
      DATA_DIR = "/content/drive/MyDrive/datasets/brain_tumor_split"
      EXTRACT_PATH = "/content/drive/MyDrive/datasets/brain_tumor_raw"
      MODEL_DIR = "/content/drive/MyDrive/models"
    else:
    # Rutime paths
      print("Using runtime and non-persistent dataset")
      RAW_DATA_DIR = "/content/datasets/brain_tumor_raw/Brain_Cancer raw MRI data/Brain_Cancer"
      DATA_DIR = "/content/datasets/brain_tumor_split"
      EXTRACT_PATH = "/content/datasets/brain_tumor_raw"
      MODEL_DIR = "/content/models"

    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(MODEL_DIR, exist_ok=True)

    # Download dataset from Kaggle if not already done
    if not os.path.exists(EXTRACT_PATH):
        !kaggle datasets download -d orvile/brain-cancer-mri-dataset
        os.makedirs(EXTRACT_PATH, exist_ok=True)
        !unzip -q brain-cancer-mri-dataset.zip -d "$EXTRACT_PATH"

else:
    print("Running locally on Mac")
    DATA_DIR = "/Users/thodorischaros/Documents/MSc/datasets/brain_tumor_split"
    MODEL_DIR = "/Users/thodorischaros/Documents/MSc/AIDL_A02_NeuralNetworksandDeepLearning/final_project/models"
    RAW_DATA_DIR = "/Users/thodorischaros/Documents/MSc/datasets/Brain_Cancer raw MRI data/Brain_Cancer"
    EXTRACT_PATH = "/Users/thodorischaros/Documents/MSc/datasets/brain_tumor_raw"

##### 3. Dataset analysis

In [ ]:
# Raw dataset distribution

plot_raw_dataset(RAW_DATA_DIR)

In [ ]:
# Split dataset to train/val/test

if not os.path.exists(os.path.join(DATA_DIR, "Training")):
    split_dataset(RAW_DATA_DIR, DATA_DIR)

train_ds, val_ds, test_ds, class_names = load_dataset_from_splits(DATA_DIR)

print("Images loaded")
print("Number of training batches:", len(train_ds))
print("Number of validation batches:", len(val_ds))
print("Number of test batches:", len(test_ds))
print("Classes:", class_names)

In [ ]:
# train/test/val datasets distribution

if 'train_ds' in globals():
    train_stats = analyze_datasets(train_ds, "Training Dataset", CLASS_NAMES)
    plot_class_distribution(train_stats, "Training Set - Class Distribution")

print("\n" + "="*60)

if 'val_ds' in globals():
    val_stats = analyze_datasets(val_ds, "Validation Dataset", CLASS_NAMES)
    plot_class_distribution(val_stats, "Validation Set - Class Distribution")

print("\n" + "="*60)

if 'test_ds' in globals():
    test_stats = analyze_datasets(test_ds, "Test Dataset", CLASS_NAMES)
    plot_class_distribution(test_stats, "Test Set - Class Distribution")

In [ ]:
# sample images from dataset

show_dataset_samples()

In [ ]:
# sample from train dataset to check that all classes were loaded and shuffled

for images, labels in train_ds.take(1):
    print("Image shape:", images.shape)
    print("Image range:", images.numpy().min(), "to", images.numpy().max())
    print("Labels:", labels.numpy()[:10])
    print("Unique labels:", np.unique(labels.numpy()))

##### 4. Custom CNNs

CNN 1


*  CNN5 with different lr and epochs



In [ ]:
def build_custom_cnn_1(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    model = models.Sequential([

        layers.Input(shape=input_shape),

        # Block 1
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Dropout(0.3),

        # Block 2
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, 3, activation='relu', padding='same'),
        layers.MaxPooling2D(),
        layers.Dropout(0.3),

        # Classifier Head
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(num_classes, activation='softmax')
    ])

    return model

In [ ]:
cnn1 = build_custom_cnn_1()
history1, best_cnn1_model_path = compile_and_fit(cnn1, train_ds, val_ds,
                                            model_name="CNN_1",
                                            epochs=40, lr=1e-3)

In [ ]:
best_cnn1_model = keras.models.load_model(best_cnn1_model_path)
evaluate_model(best_cnn1_model, test_ds, class_names, model_name="CNN1")

In [ ]:
plot_training_curves(history1, title='CNN_1')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn1.save(os.path.join(MODEL_DIR, f"cnn1_{timestamp}.h5"))
# save_model_summary(best_cnn1_model, model_name="CNN1", save_dir=os.path.join(MODEL_DIR, "summaries"))

In [ ]:
cnn5 = build_custom_cnn_1()
history5, best_cnn5_model_path = compile_and_fit(cnn5, train_ds, val_ds,
                                            model_name="CNN_5",
                                            epochs=40, lr=1e-3)

best_cnn5_model = keras.models.load_model(best_cnn5_model_path)
evaluate_model(best_cnn5_model, test_ds, class_names, model_name="CNN5")

plot_training_curves(history5, title='CNN_5')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn5.save(os.path.join(MODEL_DIR, f"cnn5_{timestamp}.h5"))
# save_model_summary(best_cnn1_model, model_name="CNN5", save_dir=os.path.join(MODEL_DIR, "summaries"))

In [ ]:
cnn4 = build_custom_cnn_1()
history4, best_cnn4_model_path = compile_and_fit(cnn4, train_ds, val_ds,
                                            model_name="CNN_4",
                                            epochs=40, lr=1e-5)

best_cnn4_model = keras.models.load_model(best_cnn4_model_path)
evaluate_model(best_cnn4_model, test_ds, class_names, model_name="CNN4")

plot_training_curves(history4, title='CNN_4')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn4.save(os.path.join(MODEL_DIR, f"cnn4_{timestamp}.h5"))
# save_model_summary(best_cnn4_model, model_name="CNN4", save_dir=os.path.join(MODEL_DIR, "summaries"))

CNN 2

In [ ]:
def build_custom_cnn_2(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)

    # Block 2
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

In [ ]:
cnn2 = build_custom_cnn_2()
history2, best_cnn2_model_path = compile_and_fit(cnn2, train_ds, val_ds,
                           model_name="CNN_2",
                           epochs=30, lr=1e-5)

In [ ]:
best_cnn2_model = keras.models.load_model(best_cnn2_model_path)
evaluate_model(best_cnn2_model, test_ds, class_names, model_name="CNN2")

In [ ]:
plot_training_curves(history2, title='CNN_2')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn2.save(os.path.join(MODEL_DIR, f"cnn2_{timestamp}.h5"))
# save_model_summary(best_cnn2_model, model_name="CNN2", save_dir=os.path.join(MODEL_DIR, "summaries"))

CNN 3

In [ ]:
def build_custom_cnn_3(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)

    # Block 2
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)

    # Block 3
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.4)(x)

    # Classifier Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

In [ ]:
cnn3 = build_custom_cnn_3()
history3, best_cnn3_model_path = compile_and_fit(cnn3, train_ds, val_ds,
                           model_name="CNN_3",
                           epochs=30, lr=1e-3)

In [ ]:
best_cnn3_model = keras.models.load_model(best_cnn3_model_path)
evaluate_model(best_cnn3_model, test_ds, class_names, model_name="CNN3")

In [ ]:
plot_training_curves(history3, title='CNN_3')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn3.save(os.path.join(MODEL_DIR, f"cnn3_{timestamp}.h5"))
# save_model_summary(best_cnn3_model, model_name="CNN3", save_dir=os.path.join(MODEL_DIR, "summaries"))

CNN 6

In [ ]:
def build_custom_cnn_6(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.4)(x)

    # Block 2
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.4)(x)

    # Block 3
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.6)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

In [ ]:
cnn6 = build_custom_cnn_6()
history6, best_cnn6_model_path = compile_and_fit(cnn6, train_ds, val_ds,
                           model_name="CNN_6",
                           epochs=30, lr=1e-3)

In [ ]:
best_cnn6_model = keras.models.load_model(best_cnn6_model_path)
evaluate_model(best_cnn6_model, test_ds, class_names, model_name="CNN6")

In [ ]:
plot_training_curves(history6, title='CNN_6')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# cnn6.save(os.path.join(MODEL_DIR, f"cnn6_{timestamp}.h5"))
# save_model_summary(best_cnn6_model, model_name="CNN6", save_dir=os.path.join(MODEL_DIR, "summaries"))

Multi Branch CNN

In [ ]:
def build_multibranch_cnn(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    # Branch 1: Fine details (small filters)
    branch1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    branch1 = layers.Conv2D(32, 3, activation='relu', padding='same')(branch1)
    branch1 = layers.MaxPooling2D(2)(branch1)
    branch1 = layers.Conv2D(64, 3, activation='relu', padding='same')(branch1)
    branch1 = layers.MaxPooling2D(2)(branch1)
    branch1 = layers.GlobalAveragePooling2D()(branch1)

    # Branch 2: Coarse features (larger filters)
    branch2 = layers.Conv2D(32, 7, activation='relu', padding='same')(inputs)
    branch2 = layers.Conv2D(32, 5, activation='relu', padding='same')(branch2)
    branch2 = layers.MaxPooling2D(2)(branch2)
    branch2 = layers.Conv2D(64, 5, activation='relu', padding='same')(branch2)
    branch2 = layers.MaxPooling2D(2)(branch2)
    branch2 = layers.GlobalAveragePooling2D()(branch2)

    # Branch 3: Edge detection
    branch3 = layers.Conv2D(16, 1, activation='relu', padding='same')(inputs)
    branch3 = layers.Conv2D(32, 3, activation='relu', padding='same')(branch3)
    branch3 = layers.Conv2D(32, 3, activation='relu', padding='same')(branch3)
    branch3 = layers.MaxPooling2D(4)(branch3)
    branch3 = layers.Conv2D(64, 3, activation='relu', padding='same')(branch3)
    branch3 = layers.GlobalAveragePooling2D()(branch3)

    # Combine all branches
    combined = layers.Concatenate()([branch1, branch2, branch3])

    # Final classification
    x = layers.Dense(256, activation='relu')(combined)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs, name='MultiBranch_CNN')

In [ ]:
build_multibranch_cnn = build_multibranch_cnn()
build_multibranch_cnn_history, build_multibranch_cnn_model_path = compile_and_fit(build_multibranch_cnn, train_ds, val_ds,
                                            model_name="build_multibranch_cnn",
                                            epochs=30, lr=1e-3)

best_build_multibranch_cnn = keras.models.load_model(build_multibranch_cnn_model_path)
evaluate_model(build_multibranch_cnn, test_ds, class_names, model_name="build_multibranch_cnn")

plot_training_curves(build_multibranch_cnn_history, title='build_multibranch_cnn')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# build_multibranch_cnn.save(os.path.join(MODEL_DIR, f"multibranch_cnn_{timestamp}.h5"))
# save_model_summary(best_build_multibranch_cnn, model_name="multibranch_cnn", save_dir=os.path.join(MODEL_DIR, "summaries"))

Multi Scale CNN

In [ ]:
def build_multiscale_cnn(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    def inner_block(x, filters):
        # 1x1 conv
        branch1 = layers.Conv2D(filters//4, 1, activation='relu', padding='same')(x)

        # 1x1 -> 3x3 conv
        branch2 = layers.Conv2D(filters//4, 1, activation='relu', padding='same')(x)
        branch2 = layers.Conv2D(filters//4, 3, activation='relu', padding='same')(branch2)

        # 1x1 -> 5x5 conv (replaced with two 3x3 for efficiency)
        branch3 = layers.Conv2D(filters//4, 1, activation='relu', padding='same')(x)
        branch3 = layers.Conv2D(filters//4, 3, activation='relu', padding='same')(branch3)
        branch3 = layers.Conv2D(filters//4, 3, activation='relu', padding='same')(branch3)

        # MaxPool -> 1x1 conv
        branch4 = layers.MaxPooling2D(3, strides=1, padding='same')(x)
        branch4 = layers.Conv2D(filters//4, 1, activation='relu', padding='same')(branch4)

        # Concatenate all branches
        return layers.Concatenate()([branch1, branch2, branch3, branch4])

    # Initial conv
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)

    # Inner blocks
    x = inner_block(x, 64)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.3)(x)

    x = inner_block(x, 128)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Dropout(0.3)(x)

    x = inner_block(x, 256)
    x = layers.GlobalAveragePooling2D()(x)

    # Classification head
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs, name='MultiScale_CNN')

In [ ]:
build_multiscale_cnn = build_multiscale_cnn()
build_multiscale_cnn_history, build_multiscale_cnn_model_path = compile_and_fit(build_multiscale_cnn, train_ds, val_ds,
                                            model_name="build_multiscale_cnn",
                                            epochs=20, lr=1e-4)

best_build_multiscale_cnn = keras.models.load_model(build_multiscale_cnn_model_path)
evaluate_model(best_build_multiscale_cnn, test_ds, class_names, model_name="build_multiscale_cnn")

plot_training_curves(build_multiscale_cnn_history, title='build_multiscale_cnn')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# build_multiscale_cnn.save(os.path.join(MODEL_DIR, f"multiscale_cnn_{timestamp}.h5"))
# save_model_summary(best_build_multiscale_cnn, model_name="multiscale_cnn", save_dir=os.path.join(MODEL_DIR, "summaries"))

##### 5. Transfer Learning

In [ ]:
@register_keras_serializable(package="preproc")
def vgg16_preproc(x): return vgg16.preprocess_input(x)

@register_keras_serializable(package="preproc")
def resnet50_preproc(x): return resnet50.preprocess_input(x)

@register_keras_serializable(package="preproc")
def inceptionv3_preproc(x): return inception_v3.preprocess_input(x)

VGG 16

In [ ]:
def build_vgg16(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    x = layers.Rescaling(255.0, name="to_255")(inputs)
    x = layers.Lambda(vgg16_preproc, name="vgg16_prep")(x)

    base = keras.applications.VGG16(include_top=False,
                                    input_shape=input_shape,
                                    weights='imagenet')
    base.trainable = False

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(256,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)

    return models.Model(base.input, outputs)

In [ ]:
vgg16_model = build_vgg16()
history_vgg16, best_vgg16_model_path = compile_and_fit(vgg16_model, train_ds, val_ds,
                              model_name="VGG16_finetune",
                              epochs=20, lr=1e-4)

In [ ]:
best_vgg16_model = keras.models.load_model(best_vgg16_model_path)
evaluate_model(best_vgg16_model, test_ds, class_names, model_name="VGG16")

In [ ]:
plot_training_curves(history_vgg16, title='VGG16')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# vgg16_model.save(os.path.join(MODEL_DIR, f"vgg16_{timestamp}.h5"))
# save_model_summary(best_vgg16_model, model_name="VGG16", save_dir=os.path.join(MODEL_DIR, "summaries"))

ResNet-50

In [ ]:
def build_resnet50(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    x = layers.Rescaling(255.0, name="to_255")(inputs)
    x = layers.Lambda(resnet50_preproc, name="resnet50_prep")(x)

    base = keras.applications.ResNet50(include_top=False,
                                       input_shape=input_shape,
                                       weights='imagenet')
    base.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dense(256,activation='relu')(x)
    outputs = layers.Dense(num_classes,activation='softmax')(x)

    return models.Model(base.input, outputs)

In [ ]:
resnet_model = build_resnet50()
history_resnet, best_resnet_model_path = compile_and_fit(resnet_model, train_ds, val_ds,
                                 model_name="ResNet50_finetune",
                                 epochs=20, lr=1e-3)

In [ ]:
best_resnet_model = keras.models.load_model(best_resnet_model_path)
evaluate_model(best_resnet_model, test_ds, class_names, model_name="RESNET50")

In [ ]:
plot_training_curves(history_resnet, title='ResNet-50')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# resnet_model.save(os.path.join(MODEL_DIR, f"resnet50_{timestamp}.h5"))
# save_model_summary(best_vgg16_model, model_name="RESNET50", save_dir=os.path.join(MODEL_DIR, "summaries"))

InceptionV3

In [ ]:
def build_inceptionv3_model(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES):

    inputs = layers.Input(shape=input_shape)

    if input_shape[:2] != (299, 299):
        x = layers.Resizing(299, 299)(inputs)
    else:
        x = inputs

    x = layers.Rescaling(255.0, name="to_255")(x)
    x = layers.Lambda(inceptionv3_preproc, name="inception_prep")(x)

    base_model = InceptionV3(
        include_top=False,
        input_shape=(299, 299, 3),  # InceptionV3 expects 299x299
        weights='imagenet',
        pooling='avg'
    )

    base_model.trainable = False

    x = base_model(x, training=False)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)

In [ ]:
inceptionv3_model = build_inceptionv3_model()
history_inceptionv3, best_inceptionv3_model_path = compile_and_fit(inceptionv3_model, train_ds, val_ds,
                                 model_name="InceptionV3_finetune",
                                 epochs=20, lr=1e-4)

In [ ]:
best_inceptionv3_model = keras.models.load_model(best_inceptionv3_model_path)
evaluate_model(best_inceptionv3_model, test_ds, class_names, model_name="INCEPTIONV3")

In [ ]:
plot_training_curves(history_inceptionv3, title='InecptioV3')

In [ ]:
# use to save model and mdel description
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# inceptionv3_model.save(os.path.join(MODEL_DIR, f"inceptionv3_{timestamp}.h5"))
# save_model_summary(best_inceptionv3_model, model_name="INCEPTIONV3", save_dir=os.path.join(MODEL_DIR, "summaries"))

##### 6. Data Augmentation

In [ ]:
# Configuration A

data_augment = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.05)
])

# Configuration B

# data_augment = keras.Sequential([
#     layers.RandomBrightness(0.03),
#     layers.RandomContrast(0.03),
# ])

aug_train_ds = train_ds.map(lambda x, y: (data_augment(x, training=True), y))

Best CNN - CNN6

In [ ]:
cnn6_aug = build_custom_cnn_6()
history_aug_cnn6, cnn6_aug_path = compile_and_fit(cnn6_aug, aug_train_ds, val_ds,
                                     model_name="Aug_CNN6",
                                     epochs=30, lr=1e-3)

In [ ]:
cnn6_aug_model = keras.models.load_model(cnn6_aug_path)
evaluate_model(cnn6_aug_model, test_ds, class_names, model_name="CNN6_aug")

In [ ]:
plot_training_curves(history_aug_cnn6, title='CNN6_aug')

VGG16

In [ ]:
vgg16_aug = build_vgg16()
history_aug_vgg16, vgg16_aug_path = compile_and_fit(vgg16_aug, aug_train_ds, val_ds,
                                  model_name="vgg16_aug",
                                  epochs=20, lr=1e-3)

In [ ]:
vgg16_aug_model = keras.models.load_model(vgg16_aug_path)
evaluate_model(vgg16_aug_model, test_ds, class_names, model_name="VGG16_aug")

In [ ]:
plot_training_curves(history_aug_vgg16, title='VGG16_aug')

ResNet-50

In [ ]:
resnet_aug = build_resnet50()
history_aug_resnet50, resnet50_aug_path = compile_and_fit(resnet_aug, aug_train_ds, val_ds,
                                     model_name="ResNet50_aug",
                                     epochs=20, lr=1e-4)

In [ ]:
resnet50_aug_model = keras.models.load_model(resnet50_aug_path)
evaluate_model(resnet50_aug_model, test_ds, class_names, model_name="RESNET50_aug")

In [ ]:
plot_training_curves(history_aug_resnet50, title='RESNET50_aug')

InceptionV3

In [ ]:
inceptionv3_aug = build_inceptionv3_model()
history_aug_inceptionv3, inceptionv3_model_path = compile_and_fit(inceptionv3_aug, aug_train_ds, val_ds,
                                 model_name="InceptionV3_aug",
                                 epochs=20, lr=1e-4)

In [ ]:
inceptionv3_aug_model = keras.models.load_model(inceptionv3_model_path)
evaluate_model(inceptionv3_aug_model, test_ds, class_names, model_name="INCEPTIONV3_aug")

In [ ]:
plot_training_curves(history_aug_inceptionv3, title='INCEPTIONV3_aug')